Copyright 2023 Louis Wendler

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

<a href="https://colab.research.google.com/github/1ucky40nc3/easy_mpt/blob/main/examples/notebooks/run_mpt_with_ggml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install Requirements
!pip install einops
!pip install transformers
!pip install accelerate
!pip install sentencepiece

!git lfs install

In [ ]:
# @title Clone the ggml Repository
!git clone https://github.com/ggerganov/ggml

In [ ]:
# @title Build the ggml MPT Features
!mkdir -p /content/ggml/build
%cd /content/ggml/build
!cmake ..
!make -j4 mpt mpt-quantize

In [ ]:
# @title Download the `mosaicml/mpt-7b-instruct` PLM
!mkdir -p /content/models
%cd /content/models
!git clone https://huggingface.co/mosaicml/mpt-7b-instruct

In [ ]:
# @title Convert the PLM into the ggml Format
%cd /content/ggml/build
!python ../examples/mpt/convert-h5-to-ggml.py /content/models/mpt-7b-instruct 0

In [ ]:
# @title Quantize the PLM
%cd /content/ggml/build
!./bin/mpt-quantize \
    /content/models/mpt-7b-instruct/ggml-model-f32.bin \
    /content/models/mpt-7b-instruct/model-quant.bin \
    q4_0

In [ ]:
# @title Do some Inference
%cd /content/ggml/build
!./bin/mpt \
    -m /content/models/mpt-7b-instruct/model-quant.bin \
    -p "Create a list of funny memes."